In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import os

In [ ]:
train_dir = "/Users/nithinparthasarathy/Downloads/HERE_Chicago_Hackathon/here-norbel/data_chicago_hackathon_2024/cnn_model/Circle-Detection-CNN/datasets/train"
test_dir = "/Users/nithinparthasarathy/Downloads/HERE_Chicago_Hackathon/here-norbel/data_chicago_hackathon_2024/cnn_model/Circle-Detection-CNN/datasets/val"

In [ ]:
# Constants
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE_TRAIN = 32
BATCH_SIZE_TEST = 20

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=360,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

# Data Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE_TRAIN,
    class_mode='binary',
    subset='training',
    shuffle=True  # Make sure shuffling is enabled
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE_TEST,
    class_mode='binary',
    subset='validation',
    shuffle=True
)

# Calculate class weights
labels = np.array(train_generator.labels)
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)
class_weight_dict = dict(enumerate(class_weights))
print("Class weights:", class_weight_dict)

# Model Architecture with adjustments for class imbalance
model = Sequential([
    # First Convolutional Block
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    # Second Convolutional Block
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    # Third Convolutional Block
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    # Use a smaller final dense layer
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    # Final layer with adjusted threshold
    Dense(1, activation='sigmoid')
])

# Compile with adjusted threshold in metrics
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=[
        tf.keras.metrics.BinaryAccuracy(threshold=0.3),  # Adjusted threshold
        tf.keras.metrics.Precision(thresholds=0.3),
        tf.keras.metrics.Recall(thresholds=0.3)
    ]
)

# Callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.00001
)

# Training with class weights
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5,
    callbacks=[early_stopping, reduce_lr],
    class_weight=class_weight_dict  # Apply class weights
)

# Custom prediction function with adjusted threshold
def predict_with_threshold(model, generator, threshold=0.3):
    predictions = model.predict(generator)
    return (predictions > threshold).astype(int)

# Evaluation with adjusted threshold
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE_TEST,
    class_mode='binary',
    shuffle=False  # Important for proper evaluation
)

# Get predictions with adjusted threshold
y_pred = predict_with_threshold(model, test_generator)
y_true = test_generator.labels

# Calculate metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)

print(f"Test accuracy: {accuracy:.4f}")
print(f"Test precision: {precision:.4f}")
print(f"Test recall: {recall:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

In [ ]:
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test precision: {test_precision:.4f}")
print(f"Test recall: {test_recall:.4f}")

In [ ]:
import random
import matplotlib.pyplot as plt

# Get the class labels (subdirectory names)
class_names = os.listdir(test_dir)

# Choose a random class (roundabout or no_roundabout)
random_class = random.choice(class_names)

# Get the path to the randomly selected class
class_path = os.path.join(test_dir, random_class)

# Get all image files in that class
image_files = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png'))]

# Pick a random image file
random_image_file = random.choice(image_files)

# Load and display the image
image_path = os.path.join(class_path, random_image_file)
print(image_path)
image = load_img(image_path, target_size=(128, 128))  # Resize to match model input size

# Convert the image to an array for display purposes
image_array = img_to_array(image)

# Show the image
plt.imshow(image_array / 255.0)  # Normalize for display
plt.axis('off')
plt.title(f"Label: {random_class}")  # Show label
print(model.predict(image_path))

plt.show()

In [ ]:
predictions = model.predict(test_generator)

# Convert probabilities to class labels
predicted_classes = (predictions > 0.5).astype("int32")
# print(predictions)

# Print the predicted values alongside the filenames
for filename, predicted_class in zip(test_generator.filenames, predicted_classes):
    print(f"Filename: {filename}, Predicted class: {'roundabout' if predicted_class == 1 else 'no roundabout'}")